In [ ]:
import pandas as pd
import glob

file_paths = glob.glob('/content/Untitled Folder/*.csv')
all_dataframes = []
for file_path in file_paths:
    df = pd.read_csv(file_path)
    all_dataframes.append(df)

vehicles_df = pd.concat(all_dataframes, ignore_index=False)

vehicles_df.head()
vehicles_df.shape

## Display precentage of null valus

In [ ]:

null_counts = vehicles_df.isnull().sum()
print(null_counts)

null_counts = df.isnull().sum()
null_percentage = (null_counts / len(df)) * 100
null_percentage = null_percentage.round(2)
print(null_percentage)


## Display the structure of the DataFrame

In [ ]:

vehicles_df.drop_duplicates(keep='first', inplace=True)
null_counts = vehicles_df.isnull().sum()
print(null_counts)
vehicles_df.isnull().sum().sum()
print(vehicles_df.info())
print(vehicles_df.describe())
print(vehicles_df.head())
print(123)
print(123)

In [ ]:
#Drop the valus that has all tha valus are null
vehicles_df.dropna(how='all', inplace=True)

# Check to confirm no null values remain
vehicles_df[['Vehicle Model Year', 'ZIP Code']] = vehicles_df[['Vehicle Model Year', 'ZIP Code']].interpolate(method='linear')

#fill the missing valus
vehicles_df=vehicles_df.fillna({'Vehicle Model':'unknown','Vehicle Make':'unknown','Vehicle Color':'unknown','Vehicle Make':'unknown','Address':'unknown','City':'unknown','State':'unknown','Taxi Affiliation':'Affiliation unknown','Taxi Medallion License Management ':'unknown'})

null_counts = vehicles_df.isnull().sum()
print(null_counts)

In [ ]:
vehicles_df.head()

## Adding new columns and Column Removal

In [ ]:
#Adding new columns to the Dataframe
vehicles_df['Vehicle Type'] = vehicles_df['Record ID'].apply(lambda x: x.split('-')[0])

#Column Removal
vehicles_df.drop(columns=['Address', 'Public Vehicle Number'], inplace=True)



In [ ]:

vehicles_df.head()

## Dashboard

In [ ]:
pip install jupyter_dash

In [ ]:
pip install Dash

### Prepare data for visualizations

In [ ]:

from dash import Dash, dcc, html
import plotly.express as px

required_columns = ['Vehicle Make', 'Wheelchair Accessible', 'Status',
                    'Vehicle Model Year', 'ZIP Code', 'Taxi Affiliation']
for col in required_columns:
    if col not in vehicles_df.columns:
        print(f"Column {col} is missing in the data.")


# Prepare data for visualizations
vehicle_make_distribution = vehicles_df['Vehicle Make'].value_counts().reset_index()
vehicle_make_distribution.columns = ['Vehicle Make', 'Count']

wheelchair_accessible = vehicles_df['Wheelchair Accessible'].value_counts().reset_index()
wheelchair_accessible.columns = ['Wheelchair Accessible', 'Count']

status_distribution = vehicles_df['Status'].value_counts().reset_index()
status_distribution.columns = ['Status', 'Count']

model_year_status = vehicles_df.groupby(['Vehicle Model Year', 'Status']).size().reset_index(name='Count')

zip_distribution = vehicles_df['ZIP Code'].value_counts().reset_index()
zip_distribution.columns = ['ZIP Code', 'Count']

taxi_affiliation = vehicles_df['Taxi Affiliation'].value_counts().reset_index()
taxi_affiliation.columns = ['Taxi Affiliation', 'Count']

license_management = vehicles_df['Taxi Medallion License Management '].value_counts().reset_index()
license_management.columns = ['License Management', 'Count']

# Filter and sort for better visualizations
vehicle_make_distribution = vehicle_make_distribution[vehicle_make_distribution['Count'] > 5]
vehicle_make_distribution.sort_values(by='Count', ascending=False, inplace=True)

zip_distribution = zip_distribution[zip_distribution['Count'] > 10]

# Create the Dash app
app = Dash(__name__)



<IPython.core.display.Javascript object>